### BiRealNet
This notebook demonstrates the binarization of ResNet Model using the BiRealNet

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [3]:
import sys
import os
import shutil
sys.path.append("../../")
sys.path.append("../")

In [4]:
# load important packages
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.datasets.classification import DatasetFactory
from datasets import DataManager

/opt/conda/envs/ml/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/envs/ml/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationESs
  warn(f"Failed to load image Python extension: {e}")


###  Define Model and Model Compression Strategy

In [5]:
from trailmet.algorithms.binarize.birealnet import BirealNet
from trailmet.models.resnet import make_birealnet50

In [6]:
data_root = './'

In [7]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(1024)  

> SEEDING DONE


### Augmentations

In [8]:
crop_scale = 0.08
lighting_param = 0.1
train_transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.RandomResizedCrop(224, scale=(crop_scale, 1.0)),
                                transforms.RandomHorizontalFlip()])
val_transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Resize(256),
                                transforms.CenterCrop(224)
                            ])
test_transform = val_transform

### Configuration Class

In [9]:
class CFG:
    def __init__(self):
        self.batch_size = 64
        self.valid_size = 0.1
        self.num_train = 0
        self.epochs = 256
        self.optimizer = torch.optim.Adam
        self.lr = 0.001
        self.momentum = 0.9
        self.weight_decay = 0
        self.save_path = './save_path_resnet50_cifar100_fullbin'
        self.data_path = ''
        self.label_smooth = 0.1
        self.workers = 4
        self.device = 'cuda'
        self.dataset = 'c100'  #valid options are c100(CIFAR100), c10(CIFAR10) and tin(TinyImageNet)
        self.num_classes = 100 #10 for CIFAR10 // 100 for CIFAR100 // 200 for Tiny ImageNet 
        self.num_fp = 0

### Define DataLoaders

In [10]:
args = CFG()
args = args.__dict__
data_object = DataManager(args)
trainloader, valloader, testloader = data_object.prepare_data(train_transform, val_transform)
dataloaders = {
        'train': trainloader, 'val': valloader, "test": testloader
}

... Preparing data ...
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
using fixed split
45000 5000


### Model

In [11]:
model = make_birealnet50(num_classes=args['num_classes'], insize=32, num_fp=args['num_fp'])

In [12]:
algo = BirealNet(model, dataloaders, **args)

### Training and Evaluating the Binarized ResNet Model

In [13]:
algo.binarize()

10/05 03:56:08 PM CFG = {'batch_size': 64, 'valid_size': 0.1, 'num_train': 0, 'epochs': 256, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.001, 'momentum': 0.9, 'weight_decay': 0, 'save_path': './save_path_resnet50_cifar100_fullbin', 'data_path': '', 'label_smooth': 0.1, 'workers': 4, 'device': 'cuda', 'dataset': 'c100', 'num_classes': 100, 'num_fp': 0}
10/05 03:56:08 PM BirealNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BinaryBasicBlock(
      (activation): BinaryActivation()
      (conv): HardBinaryConv()
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BinaryBasicBlock(
      (activation): BinaryActivation()
      (conv): HardBinaryCo

/opt/conda/envs/ml/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch: [255][  0/704]	Time 11.089 (11.089)	Data  0.948 ( 0.948)	Loss 1.6747e+00 (1.6747e+00)	Acc@1  79.69 ( 79.69)	Acc@5  92.19 ( 92.19)
Epoch: [255][  1/704]	Time  0.321 ( 5.705)	Data  0.035 ( 0.491)	Loss 1.7187e+00 (1.6967e+00)	Acc@1  71.88 ( 75.78)	Acc@5  92.19 ( 92.19)
Epoch: [255][  2/704]	Time  0.385 ( 3.931)	Data  0.002 ( 0.328)	Loss 1.8665e+00 (1.7533e+00)	Acc@1  73.44 ( 75.00)	Acc@5  90.62 ( 91.67)
Epoch: [255][  3/704]	Time  0.424 ( 3.055)	Data  0.002 ( 0.246)	Loss 1.7047e+00 (1.7411e+00)	Acc@1  79.69 ( 76.17)	Acc@5  93.75 ( 92.19)
Epoch: [255][  4/704]	Time  0.443 ( 2.532)	Data  0.009 ( 0.199)	Loss 1.9817e+00 (1.7892e+00)	Acc@1  57.81 ( 72.50)	Acc@5  87.50 ( 91.25)
Epoch: [255][  5/704]	Time  0.433 ( 2.182)	Data  0.016 ( 0.168)	Loss 1.5817e+00 (1.7546e+00)	Acc@1  81.25 ( 73.96)	Acc@5  95.31 ( 91.93)
Epoch: [255][  6/704]	Time  0.445 ( 1.934)	Data  0.001 ( 0.144)	Loss 2.0541e+00 (1.7974e+00)	Acc@1  67.19 ( 72.99)	Acc@5  87.50 ( 91.29)
Epoch: [255][  7/704]	Time  0.488 ( 1.753

Epoch: [255][ 60/704]	Time  0.421 ( 0.601)	Data  0.001 ( 0.030)	Loss 1.5880e+00 (1.8355e+00)	Acc@1  79.69 ( 72.23)	Acc@5  92.19 ( 90.22)
Epoch: [255][ 61/704]	Time  0.480 ( 0.599)	Data  0.014 ( 0.030)	Loss 1.8840e+00 (1.8363e+00)	Acc@1  65.62 ( 72.13)	Acc@5  92.19 ( 90.25)
Epoch: [255][ 62/704]	Time  0.441 ( 0.596)	Data  0.021 ( 0.030)	Loss 1.8783e+00 (1.8370e+00)	Acc@1  70.31 ( 72.10)	Acc@5  87.50 ( 90.20)
Epoch: [255][ 63/704]	Time  0.379 ( 0.593)	Data  0.004 ( 0.029)	Loss 1.8595e+00 (1.8373e+00)	Acc@1  68.75 ( 72.05)	Acc@5  93.75 ( 90.26)
Epoch: [255][ 64/704]	Time  0.431 ( 0.590)	Data  0.001 ( 0.029)	Loss 1.6604e+00 (1.8346e+00)	Acc@1  78.12 ( 72.14)	Acc@5  98.44 ( 90.38)
Epoch: [255][ 65/704]	Time  0.440 ( 0.588)	Data  0.002 ( 0.028)	Loss 1.9997e+00 (1.8371e+00)	Acc@1  67.19 ( 72.06)	Acc@5  89.06 ( 90.36)
Epoch: [255][ 66/704]	Time  0.431 ( 0.586)	Data  0.001 ( 0.028)	Loss 1.9772e+00 (1.8392e+00)	Acc@1  76.56 ( 72.13)	Acc@5  87.50 ( 90.32)
Epoch: [255][ 67/704]	Time  0.438 ( 0.583

Epoch: [255][120/704]	Time  0.413 ( 0.513)	Data  0.001 ( 0.018)	Loss 1.7644e+00 (1.8447e+00)	Acc@1  75.00 ( 71.66)	Acc@5  93.75 ( 90.47)
Epoch: [255][121/704]	Time  0.415 ( 0.512)	Data  0.002 ( 0.018)	Loss 1.8468e+00 (1.8448e+00)	Acc@1  75.00 ( 71.68)	Acc@5  93.75 ( 90.50)
Epoch: [255][122/704]	Time  0.422 ( 0.511)	Data  0.001 ( 0.017)	Loss 1.9033e+00 (1.8452e+00)	Acc@1  70.31 ( 71.67)	Acc@5  87.50 ( 90.47)
Epoch: [255][123/704]	Time  0.420 ( 0.511)	Data  0.024 ( 0.018)	Loss 1.8535e+00 (1.8453e+00)	Acc@1  68.75 ( 71.65)	Acc@5  87.50 ( 90.45)
Epoch: [255][124/704]	Time  0.421 ( 0.510)	Data  0.001 ( 0.017)	Loss 1.8848e+00 (1.8456e+00)	Acc@1  70.31 ( 71.64)	Acc@5  89.06 ( 90.44)
Epoch: [255][125/704]	Time  0.447 ( 0.510)	Data  0.001 ( 0.017)	Loss 1.9772e+00 (1.8467e+00)	Acc@1  62.50 ( 71.56)	Acc@5  90.62 ( 90.44)
Epoch: [255][126/704]	Time  0.367 ( 0.508)	Data  0.001 ( 0.017)	Loss 2.1334e+00 (1.8489e+00)	Acc@1  65.62 ( 71.52)	Acc@5  82.81 ( 90.38)
Epoch: [255][127/704]	Time  0.415 ( 0.508

...

Epoch: [255][600/704]	Time  0.465 ( 0.452)	Data  0.008 ( 0.010)	Loss 1.7630e+00 (1.8618e+00)	Acc@1  76.56 ( 71.44)	Acc@5  90.62 ( 90.36)
Epoch: [255][601/704]	Time  0.369 ( 0.452)	Data  0.018 ( 0.010)	Loss 1.9609e+00 (1.8619e+00)	Acc@1  67.19 ( 71.44)	Acc@5  87.50 ( 90.36)
Epoch: [255][602/704]	Time  0.539 ( 0.452)	Data  0.001 ( 0.010)	Loss 1.8842e+00 (1.8620e+00)	Acc@1  65.62 ( 71.43)	Acc@5  93.75 ( 90.36)
Epoch: [255][603/704]	Time  0.337 ( 0.452)	Data  0.009 ( 0.010)	Loss 1.9920e+00 (1.8622e+00)	Acc@1  70.31 ( 71.42)	Acc@5  85.94 ( 90.35)
Epoch: [255][604/704]	Time  0.447 ( 0.452)	Data  0.004 ( 0.010)	Loss 1.8535e+00 (1.8622e+00)	Acc@1  68.75 ( 71.42)	Acc@5  92.19 ( 90.36)
Epoch: [255][605/704]	Time  0.393 ( 0.452)	Data  0.002 ( 0.010)	Loss 1.8846e+00 (1.8622e+00)	Acc@1  73.44 ( 71.42)	Acc@5  90.62 ( 90.36)
Epoch: [255][606/704]	Time  0.580 ( 0.452)	Data  0.005 ( 0.010)	Loss 1.8118e+00 (1.8621e+00)	Acc@1  76.56 ( 71.43)	Acc@5  90.62 ( 90.36)
Epoch: [255][607/704]	Time  0.368 ( 0.452

Epoch: [255][660/704]	Time  0.471 ( 0.450)	Data  0.021 ( 0.010)	Loss 1.6874e+00 (1.8618e+00)	Acc@1  71.88 ( 71.41)	Acc@5  95.31 ( 90.35)
Epoch: [255][661/704]	Time  0.420 ( 0.450)	Data  0.001 ( 0.010)	Loss 1.7402e+00 (1.8616e+00)	Acc@1  73.44 ( 71.41)	Acc@5  90.62 ( 90.35)
Epoch: [255][662/704]	Time  0.449 ( 0.450)	Data  0.011 ( 0.010)	Loss 1.7537e+00 (1.8615e+00)	Acc@1  79.69 ( 71.42)	Acc@5  92.19 ( 90.36)
Epoch: [255][663/704]	Time  0.505 ( 0.450)	Data  0.002 ( 0.010)	Loss 1.9238e+00 (1.8616e+00)	Acc@1  68.75 ( 71.42)	Acc@5  89.06 ( 90.35)
Epoch: [255][664/704]	Time  0.396 ( 0.450)	Data  0.002 ( 0.010)	Loss 1.9359e+00 (1.8617e+00)	Acc@1  68.75 ( 71.42)	Acc@5  90.62 ( 90.35)
Epoch: [255][665/704]	Time  0.459 ( 0.450)	Data  0.003 ( 0.010)	Loss 2.0791e+00 (1.8620e+00)	Acc@1  64.06 ( 71.41)	Acc@5  84.38 ( 90.35)
Epoch: [255][666/704]	Time  0.463 ( 0.450)	Data  0.001 ( 0.010)	Loss 1.9763e+00 (1.8622e+00)	Acc@1  70.31 ( 71.40)	Acc@5  90.62 ( 90.35)
Epoch: [255][667/704]	Time  0.451 ( 0.450

Test: [21/79]	Time  0.238 ( 0.304)	Loss 1.7226e+00 (1.6144e+00)	Acc@1  54.69 ( 57.60)	Acc@5  79.69 ( 83.66)
Test: [22/79]	Time  0.200 ( 0.300)	Loss 1.2849e+00 (1.6001e+00)	Acc@1  70.31 ( 58.15)	Acc@5  84.38 ( 83.70)
Test: [23/79]	Time  0.245 ( 0.297)	Loss 1.5924e+00 (1.5998e+00)	Acc@1  53.12 ( 57.94)	Acc@5  81.25 ( 83.59)
Test: [24/79]	Time  0.240 ( 0.295)	Loss 1.6528e+00 (1.6019e+00)	Acc@1  62.50 ( 58.12)	Acc@5  78.12 ( 83.38)
Test: [25/79]	Time  0.207 ( 0.292)	Loss 1.9397e+00 (1.6149e+00)	Acc@1  54.69 ( 57.99)	Acc@5  75.00 ( 83.05)
Test: [26/79]	Time  0.237 ( 0.290)	Loss 1.4536e+00 (1.6089e+00)	Acc@1  62.50 ( 58.16)	Acc@5  89.06 ( 83.28)
Test: [27/79]	Time  0.193 ( 0.286)	Loss 1.2389e+00 (1.5957e+00)	Acc@1  71.88 ( 58.65)	Acc@5  87.50 ( 83.43)
Test: [28/79]	Time  0.200 ( 0.283)	Loss 2.0957e+00 (1.6129e+00)	Acc@1  39.06 ( 57.97)	Acc@5  76.56 ( 83.19)
Test: [29/79]	Time  0.231 ( 0.282)	Loss 1.7750e+00 (1.6183e+00)	Acc@1  48.44 ( 57.66)	Acc@5  76.56 ( 82.97)
Test: [30/79]	Time  0.192 ( 

Test: [ 18/157]	Time  0.288 ( 0.398)	Loss 1.3269e+00 (1.6026e+00)	Acc@1  64.06 ( 58.88)	Acc@5  93.75 ( 84.29)
Test: [ 19/157]	Time  0.284 ( 0.392)	Loss 1.4184e+00 (1.5934e+00)	Acc@1  60.94 ( 58.98)	Acc@5  90.62 ( 84.61)
Test: [ 20/157]	Time  0.240 ( 0.385)	Loss 1.6263e+00 (1.5950e+00)	Acc@1  59.38 ( 59.00)	Acc@5  81.25 ( 84.45)
Test: [ 21/157]	Time  0.308 ( 0.381)	Loss 1.3692e+00 (1.5847e+00)	Acc@1  68.75 ( 59.45)	Acc@5  87.50 ( 84.59)
Test: [ 22/157]	Time  0.235 ( 0.375)	Loss 1.6107e+00 (1.5858e+00)	Acc@1  64.06 ( 59.65)	Acc@5  87.50 ( 84.71)
Test: [ 23/157]	Time  0.285 ( 0.371)	Loss 1.5801e+00 (1.5856e+00)	Acc@1  56.25 ( 59.51)	Acc@5  85.94 ( 84.77)
Test: [ 24/157]	Time  0.267 ( 0.367)	Loss 1.8059e+00 (1.5944e+00)	Acc@1  53.12 ( 59.25)	Acc@5  84.38 ( 84.75)
Test: [ 25/157]	Time  0.273 ( 0.363)	Loss 1.6558e+00 (1.5968e+00)	Acc@1  54.69 ( 59.07)	Acc@5  84.38 ( 84.74)
Test: [ 26/157]	Time  0.374 ( 0.364)	Loss 1.7822e+00 (1.6036e+00)	Acc@1  50.00 ( 58.74)	Acc@5  78.12 ( 84.49)
Test: [ 27

Test: [ 93/157]	Time  0.291 ( 0.304)	Loss 1.8354e+00 (1.5795e+00)	Acc@1  51.56 ( 58.78)	Acc@5  81.25 ( 84.11)
Test: [ 94/157]	Time  0.267 ( 0.304)	Loss 1.7724e+00 (1.5815e+00)	Acc@1  50.00 ( 58.68)	Acc@5  84.38 ( 84.11)
Test: [ 95/157]	Time  0.285 ( 0.304)	Loss 1.5760e+00 (1.5815e+00)	Acc@1  59.38 ( 58.69)	Acc@5  84.38 ( 84.11)
Test: [ 96/157]	Time  0.252 ( 0.303)	Loss 1.5372e+00 (1.5810e+00)	Acc@1  57.81 ( 58.68)	Acc@5  81.25 ( 84.09)
Test: [ 97/157]	Time  0.314 ( 0.303)	Loss 1.8120e+00 (1.5834e+00)	Acc@1  59.38 ( 58.69)	Acc@5  78.12 ( 84.02)
Test: [ 98/157]	Time  0.262 ( 0.303)	Loss 1.5999e+00 (1.5835e+00)	Acc@1  57.81 ( 58.68)	Acc@5  85.94 ( 84.04)
Test: [ 99/157]	Time  0.302 ( 0.303)	Loss 1.2734e+00 (1.5804e+00)	Acc@1  64.06 ( 58.73)	Acc@5  85.94 ( 84.06)
Test: [100/157]	Time  0.251 ( 0.303)	Loss 1.4198e+00 (1.5788e+00)	Acc@1  60.94 ( 58.76)	Acc@5  85.94 ( 84.08)
Test: [101/157]	Time  0.317 ( 0.303)	Loss 1.8946e+00 (1.5819e+00)	Acc@1  59.38 ( 58.76)	Acc@5  79.69 ( 84.04)
Test: [102

### Loading the Checkpoint to know the test accuracy

In [ ]:
chk=torch.load('./save_path_resnet50_cifar100_fullbin/c100-model_best.pth.tar')

In [21]:
chk['epoch']

250

### Test Accuracy

In [22]:
chk['best_top1_acc']

tensor(60.9000)